In [12]:
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import style
style.use('ggplot')
import numpy as np
from sklearn.cluster import KMeans
from sklearn import preprocessing


In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
# load dataset
df = pd.read_csv("adult.xls", header=None)
for i in range(14):
  df.drop(df.loc[df[i]=="?"].index, inplace=True)
  #df = df.reset_index(drop=True)
#dataset = df.values
df.dropna(inplace=True)

Dataset=df.values

In [4]:
# encode class values as integers
Y = Dataset[:,14]
#Y =Y[1:]
print (Y)
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)
print(encoded_Y)

['income' '<=50K' '<=50K' ... '>50K' '<=50K' '<=50K']
[2 0 0 ... 1 0 0]


In [5]:
def handle_non_numerical_data(df):
    columns = df.columns.values
    for column in columns:
        text_digit_vals = {}
        def convert_to_int(val):
            return text_digit_vals[val]

        if df[column].dtype != np.int64 and df[column].dtype != np.float64:
            column_contents = df[column].values.tolist()
            unique_elements = set(column_contents)
            x = 0
            for unique in unique_elements:
                if unique not in text_digit_vals:
                    text_digit_vals[unique] = x
                    x+=1

            df[column] = list(map(convert_to_int, df[column]))

    return df

In [6]:
df = handle_non_numerical_data(df)
df=df.values

In [7]:
# split into input (X) and output (Y) variables which has 15 rows
X = df[:,0:14]
#Y = df[:,14]
print(X)

[[   56     4  4851 ...    87    39    38]
 [   65     7 18568 ...    89    73    10]
 [   68     7 14718 ...    28     7    10]
 ...
 [   27     7 16207 ...    54     7    10]
 [   57     7  8754 ...    54     7    10]
 [   71     7  7690 ...    54    44    10]]


In [8]:
# baseline model
# def create_baseline():
# 	# create model
# 	model = Sequential()
# 	model.add(Dense(14, input_dim=14, activation='relu'))
# 	model.add(Dense(1, activation='sigmoid'))
# 	# Compile model
# 	model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# 	return model

In [9]:

# larger model
def create_model():
	# create model
	model = Sequential()
	model.add(Dense(14, input_dim=14, activation='swish'))
	model.add(Dense(7, activation='relu'))
	model.add(Dense(1, activation='sigmoid'))
	# Compile model
	model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model




In [10]:
estimators = []
estimators.append(('standardize', StandardScaler()))
Model = create_model()
estimators.append(('mlp', KerasClassifier(build_fn=create_model, epochs=300, batch_size=12, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True)
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
print("Larger: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))
Model.save("Saved-Model")

C:\Users\Mr\AppData\Local\Temp/ipykernel_111120/3981285114.py:4: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead.
  estimators.append(('mlp', KerasClassifier(build_fn=create_model, epochs=300, batch_size=12, verbose=0)))
E:\Anaconda\envs\notebook\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(


Larger: 82.66% (0.80%)
INFO:tensorflow:Assets written to: Saved-Model\assets


In [11]:
#model = keras.models.load_model('path/to/location')